In [1]:
import pandas as pd
import os
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
%matplotlib inline
from gensim.models import TfidfModel
from importlib_metadata import version

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wangqiang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<strong> Package Version

In [2]:
print(version('gensim'))
print(version('nltk'))
print(version('spacy'))
print(version('pyLDAvis'))

3.8.1
3.3
2.2.1
2.1.2


## 1. Import DATA and concat the 10 years file

In [3]:
df = pd.DataFrame()
for i in range(10):
    for y in ['a']:
        path = os.getcwd()
        df_original = pd.read_csv(path+'/Dataset_FAA/'+'sdr201{}'.format(i) + y + '.txt',sep=r'\t', engine='python', error_bad_lines=False)
        df_original = df_original[df_original.c260=='A']# filter the records for part121 by matching 'A' in c260 column
        df = pd.concat([df,df_original],axis = 0)

Skipping line 36549: Expected 86 fields in line 36549, saw 90. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.


In [4]:
len(df)

553987

In [5]:
df.index = range(len(df))

## 2. Data Manipulation

* Records are sorted by the date of occurence, which is the value in c20 column, and generate two more column year and month indicating occurence year and month correspondingly

In [6]:
df.dtypes.c20

dtype('int64')

In [7]:
occcur_year = [int(x) for x in round(df.c20 /10000)]
occur_year = pd.DataFrame(occcur_year,columns = ['occur_year'])

In [8]:
occcur_month = [int(x) for x in round(df.c20 /100)]
occcur_month = [x%100 for x in occcur_month]
occur_month = pd.DataFrame(occcur_month,columns = ['occur_month'])

In [9]:
occur_month.occur_month.value_counts()
    

3     53044
1     50566
4     48749
2     47785
5     47084
10    46500
8     45196
6     44733
7     43943
9     43908
11    42328
12    40151
Name: occur_month, dtype: int64

* the column type c18 is messy, including both numeric and string. This column is segment code, so we convert all of them to numeric type.

In [10]:
df.c18 = pd.to_numeric(df.c18)

* Combine the column c120: Aircraft manufacturer's name, c130:FAA assigned code to identify aircraft group as manufacture_model

In [11]:
df[df.c120.isnull()].c130 # If the manufacture's name is null, the assigned code is also none

15        NaN
179100    NaN
396664    NaN
467900    NaN
504992    NaN
505231    NaN
544094    NaN
544095    NaN
Name: c130, dtype: object

In [12]:
df[df.c130.isnull()].c120# If the assigned code is null, the manufacture's name is also none except one record

15           NaN
100847    BOEING
179100       NaN
396664       NaN
467900       NaN
504992       NaN
505231       NaN
544094       NaN
544095       NaN
Name: c120, dtype: object

* the column type c120 and c130 are messy, including both numeric and string. This column is segment code, so we convert all of them to string type.

In [13]:
for i in range(len(df)):
    if type(df.c120[i]) != str:
        df.c120[i]=str(df.c120[i])
for i in range(len(df)):
    if type(df.c130[i]) != str:
        df.c130[i]=str(df.c130[i])

/Users/wangqiang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/wangqiang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
manu_model = []
for i in range(len(df)):
    manu_model.append(df.c120[i].strip()+' '+df.c130[i].strip())

In [15]:
manu_model = pd.DataFrame(manu_model,columns = ['manu_model'])

* Let us check what types of aircraft models in the dataset for airbus and boeing company

In [16]:
Airbus = []
Boeing = []
for i in range(len(manu_model)):
    if 'AIRBUS' in manu_model.manu_model[i]:
        Airbus.append(manu_model.manu_model[i])
    if 'BOEING' in manu_model.manu_model[i]:
        Boeing.append(manu_model.manu_model[i])

In [17]:
set(Boeing)

{'BOEING 100',
 'BOEING 234',
 'BOEING 707',
 'BOEING 717',
 'BOEING 727',
 'BOEING 737',
 'BOEING 747',
 'BOEING 75',
 'BOEING 757',
 'BOEING 767',
 'BOEING 77',
 'BOEING 777',
 'BOEING 777F',
 'BOEING 787',
 'BOEING nan'}

For aircraft models of Boeing company, we combine the identical models of Boeing 77,Boeing 777 and Boeing 777f as Boeing 777 and Boeing 75 Boeing 757 as Boeing 757

In [18]:
set(Airbus)

{'AIRBUS 300',
 'AIRBUS 310',
 'AIRBUS 320',
 'AIRBUS 330',
 'AIRBUS A300',
 'AIRBUS A318',
 'AIRBUS A319',
 'AIRBUS A320',
 'AIRBUS A321',
 'AIRBUS A330',
 'AIRBUS A350'}

For aircraft models of Airbus company, we replace the models ID with ID without A, such as replacing 'AIRBUS A300' as 'AIRBUS 300', and combine the identical models of AIRBUS A318, AIRBUS A319 and AIRBUS A320 as AIRBUS A321 as AIRBUS A320. 

In [19]:
manu_model = manu_model.replace({'BOEING 77':'BOEING 777','BOEING 777F':'BOEING 777','BOEING 75':'BOEING 757',\
                                'AIRBUS A300':'AIRBUS 300','AIRBUS A318':'AIRBUS 320','AIRBUS A319':'AIRBUS 320',\
                                'AIRBUS A321':'AIRBUS 320','AIRBUS A321':'AIRBUS A320','AIRBUS A330':'AIRBUS 330',\
                                'AIRBUS A350':'AIRBUS 350','AIRBUS A320':'AIRBUS 320'})

In [20]:
Airbus = []
Boeing = []
for i in range(len(manu_model)):
    if 'AIRBUS' in manu_model.manu_model[i]:
        Airbus.append(manu_model.manu_model[i])
    if 'BOEING' in manu_model.manu_model[i]:
        Boeing.append(manu_model.manu_model[i])

In [21]:
set(Boeing)

{'BOEING 100',
 'BOEING 234',
 'BOEING 707',
 'BOEING 717',
 'BOEING 727',
 'BOEING 737',
 'BOEING 747',
 'BOEING 757',
 'BOEING 767',
 'BOEING 777',
 'BOEING 787',
 'BOEING nan'}

In [22]:
set(Airbus)

{'AIRBUS 300', 'AIRBUS 310', 'AIRBUS 320', 'AIRBUS 330', 'AIRBUS 350'}

### CONCAT

In [23]:
df= pd.concat([df,occur_year,occur_month,manu_model], axis = 1)

In [24]:
df = df.sort_values(by='occur_year')

In [25]:
df.head()

,c5,c10,c12,c14,c15,c16,c17,c18,c20,c25,...,c616,c620,c640,c652,c654,c660,eof,occur_year,occur_month,manu_model
0,20100420,NaN,CE,2010,4,20,19MA201004136468,1,20100411,A,...,A24CE,None,None,None,None,NaN,NaN,2010,4,BEECH 1900
36841,20100511,NaN,NM,2010,5,11,SWAA102658,1,20100506,A,...,A16WE,None,None,None,None,NaN,NaN,2010,5,BOEING 737
36842,20100511,NaN,NM,2010,5,11,SWAA102659,1,20100506,A,...,A16WE,None,None,None,None,NaN,NaN,2010,5,BOEING 737
36843,20100511,NaN,NM,2010,5,11,SWAA102660,1,20100506,A,...,A16WE,None,None,None,None,NaN,NaN,2010,5,BOEING 737
36844,20100511,NaN,NM,2010,5,11,SWAA102661,1,20100506,A,...,A16WE,None,None,None,None,NaN,NaN,2010,5,BOEING 737


## 2. Basic LDA Model

In [26]:

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuations

def remove_stopwords(texts):
    stop_words = stopwords.words('english')
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def make_bigrams(texts):

# Build the bigram and trigram models
    bigram = gensim.models.Phrases(texts, min_count=3, threshold=75) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[texts], threshold=75)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)


    return [trigram[bigram[doc]] for doc in texts]

# Do lemmatization keeping only noun, adj, vb, adv
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
   
    # Initialize spacy 'en' model, keeping only tagger component (for efficiency) extract 
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [27]:
topic = {}
vis = {}
# to filter records for part 121, which is A in c260 columns
df = df[df.c260=='A']

list1 = []
list1.append('c490')
for i in ['a','b','c','d','e']:
    list1.append('c510{}'.format(i))
text = ''
for i in list1: 
    text = text+df[i].fillna('')
text = text.tolist()

# tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.
data_words = list(sent_to_words(text))

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary/ID
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [28]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=50000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Compute Perplexity
print('\nPerplexity of year 10-19 dataset:' , lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score year 10-19 dataset:', coherence_lda)

# Print the Keyword in the 20 topics
topic['10-19'] = lda_model.print_topics()
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()
vis['10-19'] = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)


Perplexity of year 10-19 dataset: -5.632007696337756

Coherence Score year 10-19 dataset: 0.5455520449509738


In [29]:
topic_df = pd.DataFrame(topic['10-19'], columns=['tpc_id', 'Topic'])#can search the dictionary

In [30]:
get_document_topics = lda_model.get_document_topics(corpus)
print(get_document_topics[0])# get_document_topics extract the topic id for each text records

[(4, 0.09119132), (11, 0.7325219), (12, 0.17052834)]


In [31]:
from operator import itemgetter
clus=[]
for i in range(len(df)):
    x = max(get_document_topics[i],key=itemgetter(1))[0]
    clus.append(x)

* output dataframe with columns: topic_id, text report, occurence year, occurence month, topic keywords,c12:region code, c18: segment code, c90:Descriptive name of part, c120: Aircraft manufacturer's name, c130:FAA assigned code to identify aircraft group, 'manu_model' :combined columns of c120+c130 ,c152 Region responsible for aircraft, c240 Location on aircraft of the defective or malfunctioning part,c250 Text reflecting condition of failed part,c501f Aircraft weight class code,c602 Aircraft wing type code

In [32]:
clustered = pd.DataFrame(list(zip(clus, text, df.occur_year, df.occur_month,df.c12,df.c18,df.c90,\
                                  df.c120,df.c130,df.manu_model,df.c152,df.c240,df.c250,df.c510f,df.c602)), 
               columns =['tpc_id', 'text', 'year','month','region','segment','name_of_part','manu_name',\
                        'model','manu_model','region_for_aircraft','defective_location','text_failed_part','weight_class',\
                        'wing_type']) 
clustered = pd.merge(clustered, topic_df, on='tpc_id', how='left')

In [107]:
clustered.head(10)

,tpc_id,text,year,month,region,segment,name_of_part,manu_name,model,manu_model,region_for_aircraft,defective_location,text_failed_part,weight_class,wing_type,Topic
0,3,"ON DESCENT TO LAND, PILOT PUT LANDING GEAR HAN...",2010,4,CE,1,RELAY,BEECH,1900,BEECH 1900,CE,LANDING GEAR,STUCK,2.0,L,"0.060*""fault"" + 0.050*""test"" + 0.043*""ground"" ..."
1,14,"DURING SCHEDULED 14Y CHECK, FOUND SKIN DEFORME...",2010,5,NM,1,SKIN,BOEING,737,BOEING 737,NM,FUSELAGE,DEFORMED,2.0,L,"0.172*""crack"" + 0.086*""skin"" + 0.063*""wing"" + ..."
2,5,"DURING SCHEDULED 14Y CHECK, FOUND FWD CARGO FR...",2010,5,NM,1,FRAME,BOEING,737,BOEING 737,NM,FUSELAGE,CRACKED,2.0,L,"0.204*""corrode"" + 0.100*""web"" + 0.070*""shear"" ..."
3,0,"DURING SCHEDULED 14Y CHECK, FOUND L1 DOOR CUTO...",2010,5,NM,1,ANGLE,BOEING,737,BOEING 737,NM,FUSELAGE,CRACKED,2.0,L,"0.073*""hole"" + 0.070*""fit"" + 0.061*""fuel"" + 0...."
4,14,"DURING SCHEDULED 14Y CHECK, FOUND STRINGER CLI...",2010,5,NM,1,STRINGER CLIP,BOEING,737,BOEING 737,NM,FUSELAGE,CRACKED,2.0,L,"0.172*""crack"" + 0.086*""skin"" + 0.063*""wing"" + ..."
5,5,"DURING SCHEDULED 14Y CHECK, FOUND AFT LOWER RT...",2010,5,NM,1,ANGLE,BOEING,737,BOEING 737,NM,FUSELAGE,CORRODED,2.0,L,"0.204*""corrode"" + 0.100*""web"" + 0.070*""shear"" ..."
6,13,AFT GALLEY EMERGENCY EXIT SIGN LENS BROKEN. R...,2010,5,NM,1,SIGN,BOEING,737,BOEING 737,NM,CABIN,BROKEN,2.0,L,"0.154*""exit"" + 0.098*""emergency"" + 0.086*""sign..."
7,17,L1 DOOR EMERGENCY EVAC SLIDE INADVERTENTLY FEL...,2010,5,NaN,4,SLIDE,BOEING,737,BOEING 737,NM,L1 DOOR,OUT OF POSITION,2.0,L,"0.211*""slide"" + 0.058*""position"" + 0.049*""loos..."
8,15,DIVERSION / DECLARED EMERGENCY. CABIN ALTITUD...,2010,5,NM,1,OUTFLOW VALVE,BOEING,737,BOEING 737,NM,CABIN PRESSURE,MALFUNCTIONED,2.0,L,"0.043*""engine"" + 0.019*""valve"" + 0.017*""ground..."
9,11,DIVERSION / DECLARED EMERGENCY. DC FAIL LIGHT...,2010,5,NM,1,BATTERY,BOEING,737,BOEING 737,NM,E&E BAY,FAULTY,2.0,L,"0.124*""emergency"" + 0.111*""light"" + 0.102*""bat..."


* Output

In [34]:
clustered.to_csv("Topic_records_clusterd.csv",index = False)
topic_df.to_csv('topic.csv',index = False)

## 3. Basic Trend of Phrase and Abbreviation 

### Phrase Trend

In [35]:
df_year = list(clustered.year)

In [36]:
text_per_year = {}
for i in range(10):
    a = []
    for j in list(clustered[clustered.year == 2010+i].index):
        a.append(texts[j])
    text_per_year[2010+i] = a

* Create Bigram list

In [37]:
phrase = []
phrase_id = []
for i in corpus:
    for j in i:
        if '_' in id2word[j[0]]:
            phrase.append(id2word[j[0]])
            phrase_id.append(j[0])

<strong> Phrase Frequency by counting the total occurence

In [38]:
#Drop duplicate
phrase = list(set(phrase))
phrase_id = list(set(phrase_id))

In [39]:
len(phrase)

4436

In [40]:
len(phrase_id)

4436

In [41]:
dic = {}
dic['phrase_id']=phrase_id
dic['phrase_name']=phrase

* Create Trend Table

In [42]:
fre_phrase = pd.DataFrame(dic, columns = ['phrase_id','phrase_name'])

In [43]:
ratio_phrase = fre_phrase

In [44]:
for i in range(10):
    fre_phrase_list = []
    phrase_ratio = []
    for y in phrase:
        fre = 0
        for z in text_per_year[2010+i]:
            for a in list(set(z)):
                if a == y:
                    fre = fre + 1
        fre_phrase_list.append(fre)
        ratio = fre/len(text_per_year[2010+i])*10000
        phrase_ratio.append(ratio)
    fre_phrase = pd.concat([fre_phrase, pd.DataFrame(fre_phrase_list,columns = [2010+i])],axis = 1)
    ratio_phrase = pd.concat([ratio_phrase, pd.DataFrame(phrase_ratio,columns = [2010+i])],axis = 1)

In [45]:
total = []
for i in range(fre_phrase.shape[0]):
    total.append(sum(fre_phrase.iloc[i][2:fre_phrase.shape[1]]))

In [46]:
fre_phrase = pd.concat([fre_phrase,pd.DataFrame(total,columns = ['total'])],axis = 1)

* phrase trend with frequency descending

In [47]:
fre_phrase.sort_values(by=['total'],ascending=False).head(20)

,phrase_id,phrase_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
3532,14699,approved_return,938,585,403,608,645,343,343,142,115,36,4158
2915,5245,assist_bottle,192,187,160,130,244,200,247,354,351,143,2208
2747,21300,first_officer,85,140,264,218,245,158,139,176,197,41,1663
3848,15285,horizontal_stabilizer,62,81,270,163,248,252,182,109,106,25,1498
990,1810,circuit_breaker,104,125,126,166,117,65,116,208,158,13,1198
3492,6413,without_incident,266,221,90,116,90,108,94,104,89,16,1194
2365,12391,could_duplicate,132,151,124,132,84,84,103,137,151,41,1139
1491,2661,corrective_action,27,38,22,12,33,28,27,89,332,411,1019
866,1607,master_caution,41,51,79,94,111,100,154,156,134,43,963
1850,11524,wheel_well,75,65,94,127,10,46,125,180,145,29,896


* phrase frequency ratio table corresponding to the above table（ratio*10000)

In [48]:
index_sorted = list(fre_phrase.sort_values(by=['total'],ascending=False).index)

In [49]:
ratio_phrase.reindex(index_sorted).head(20)

,phrase_id,phrase_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
3532,14699,approved_return,169.709251,85.235965,60.204966,87.139725,110.367721,62.162456,61.409005,26.115423,22.159277,20.423214
2915,5245,assist_bottle,34.737928,27.246368,23.902716,18.631849,41.751510,36.246330,44.221645,65.104646,67.633967,81.125546
2747,21300,first_officer,15.378770,20.398351,39.439481,31.244178,41.922623,28.634601,24.885865,32.368411,37.959805,23.259772
3848,15285,horizontal_stabilizer,11.217456,11.801903,40.335833,23.361472,42.435961,45.670376,32.584370,20.046346,20.425073,14.182788
990,1810,circuit_breaker,18.816377,18.212813,18.823389,23.791438,20.020191,11.780057,20.768060,38.253577,30.444920,7.375050
3492,6413,without_incident,48.126504,32.200254,13.445278,16.625342,15.400147,19.573018,16.829290,19.126789,17.149354,9.076984
2365,12391,could_duplicate,23.882325,22.001078,18.524605,18.918493,14.373471,15.223459,18.440605,25.195866,29.096094,23.259772
1491,2661,corrective_action,4.885021,5.536695,3.286623,1.719863,5.646721,5.074486,4.833945,16.368117,63.972869,233.165031
866,1607,master_caution,7.417995,7.430828,11.801966,13.472260,18.993515,18.123165,27.571390,28.690183,25.820375,24.394395
1850,11524,wheel_well,13.569503,9.470663,14.042846,18.201883,1.711127,8.336656,22.379375,33.104057,27.939958,16.452034


<strong> Output

In [50]:
fre_phrase.sort_values(by=['total'],ascending=False).to_csv('phrase_fre.csv')

In [51]:
ratio_phrase.reindex(index_sorted).to_csv('phrase_ratio.csv')

### Abbreviation Trend

* Abbreviation Extraction

In [52]:
abbr_list= pd.read_csv('abbr_list.csv',header = 0)

* Create Abbreviation list

In [53]:
df_year = list(clustered.year)

In [54]:
abbr_list2 = list(abbr_list.Name.str.lower())

In [55]:
abbr = []
abbr_id = []
for i in corpus:
    for j in i:
        if id2word[j[0]] in abbr_list2:
            abbr.append(id2word[j[0]])
            abbr_id.append(j[0])

In [56]:
len(abbr)

230317

In [57]:
#Drop duplicate
abbr = list(set(abbr))
abbr_id = list(set(abbr_id))

In [58]:
dic = {}
dic['abbr_id']=abbr_id
dic['abbr_name']=abbr

* Create Trend Table

In [59]:
fre_abbr = pd.DataFrame(dic, columns = ['abbr_id','abbr_name'])
ratio_abbr = fre_abbr

In [60]:
for i in range(10):
    fre_abbr_list = []
    abbr_ratio = []
    for y in abbr:
        fre = 0
        for z in text_per_year[2010+i]:
            for a in list(set(z)):
                if a == y:
                    fre = fre+ 1
        fre_abbr_list.append(fre)
        ratio = fre/len(text_per_year[2010+i])*10000
        abbr_ratio.append(ratio)
    fre_abbr = pd.concat([fre_abbr, pd.DataFrame(fre_abbr_list,columns = [2010+i])],axis = 1)
    ratio_abbr = pd.concat([ratio_abbr, pd.DataFrame(abbr_ratio,columns = [2010+i])],axis = 1)

In [61]:
total = []
for i in range(fre_abbr.shape[0]):
    total.append(sum(fre_abbr.iloc[i][2:fre_abbr.shape[1]]))

In [62]:
fre_abbr = pd.concat([fre_abbr,pd.DataFrame(total,columns = ['total'])],axis = 1)

* abbr trend with frequency descending

In [63]:
fre_abbr.sort_values(by=['total'],ascending=False).head(20)

,abbr_id,abbr_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
347,1799,srm,10916,11887,11391,13874,11218,10651,11097,10001,8812,3081,102928
53,4369,area,3673,4302,4039,3484,2836,2746,3048,2137,2342,1020,29627
165,9050,acft,563,677,1303,2334,2581,2887,2393,1384,984,27,15133
156,11056,ea,2082,2276,1859,1286,1013,1168,802,757,1065,471,12779
37,190,fig,1136,1419,1094,1274,925,1183,1533,920,642,230,10356
369,12196,air,845,1029,1010,898,690,648,738,691,797,277,7623
84,16834,pass,330,398,406,570,398,446,615,655,589,200,4607
309,5683,fire,437,515,447,453,326,324,343,415,421,172,3853
96,2587,arm,237,355,351,471,369,335,438,647,484,141,3828
271,11578,ad,281,456,524,541,319,310,370,231,401,219,3652


* phrase frequency ratio table corresponding to the above table（ratio*10000)

In [64]:
index_sorted = list(fre_abbr.sort_values(by=['total'],ascending=False).index)

In [65]:
ratio_abbr.reindex(index_sorted).head(20)

,abbr_id,abbr_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
347,1799,srm,1974.995929,1731.965672,1701.723983,1988.448254,1919.542787,1930.298307,1986.751410,1839.298194,1697.978689,1747.886765
53,4369,area,664.543793,626.812175,603.394186,499.333553,485.275748,497.662112,545.698684,393.018722,451.278494,578.657741
165,9050,acft,101.861736,98.640596,194.657743,334.513350,441.641998,523.215774,428.430758,254.533417,189.606336,15.317411
156,11056,ea,376.689403,331.618901,277.719681,184.311983,173.337212,211.678568,143.586071,139.220951,205.214174,267.203722
37,190,fig,205.532739,206.751854,163.434820,182.592120,158.279290,214.397042,274.460657,169.198514,123.706573,130.481647
369,12196,air,152.883067,149.927877,150.885894,128.703080,118.067795,117.438109,132.127831,127.082797,153.573424,157.145288
84,16834,pass,59.705813,57.989597,60.653142,81.693492,68.102873,80.829316,110.106526,120.461986,113.494036,113.462302
309,5683,fire,79.064971,75.036790,66.778213,64.924828,55.782755,58.719055,61.409005,76.323243,81.122223,97.577580
96,2587,arm,42.879629,51.724389,52.436583,67.504622,63.140603,60.712603,78.417331,118.990694,93.261653,79.990923
271,11578,ad,50.840405,66.440342,78.281395,77.537156,54.584966,56.181812,66.242950,42.483540,77.268436,124.241221


<strong>Output

In [66]:
fre_abbr.sort_values(by=['total'],ascending=False).to_csv('abbr_fre.csv')
ratio_abbr.reindex(index_sorted).to_csv('abbr_ratio.csv')

## 4. Stopwords Optimization Based on Term Frequency and Delete not issue-related words

In [106]:
# build unique word list
ttl_word_list = []
for wordlist in data_lemmatized:
    for word in wordlist:
        ttl_word_list.append(word)
unique_word_list = []
for word in ttl_word_list:
    if word not in unique_word_list:
        unique_word_list.append(word)     

In [68]:
# create bag of word
bagOfWords = []
for i in data_lemmatized:
    for w in i:
        bagOfWords.append(w)

In [69]:
# create the word count dictionary
numOfWords = dict.fromkeys(unique_word_list, 0)
for word in bagOfWords:
    numOfWords[word] += 1

In [70]:
# term frequency in python
# The number of times a word appears in a document divded by the total number of words in the document. Every document has its own term frequency.
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [71]:
tf = computeTF(numOfWords, bagOfWords)
sort_tf = dict([(i,tf[i]) for i in sorted(tf, key=tf.get, reverse=True)])
tf_df = pd.DataFrame(list(sort_tf.items()), columns=['word', 'tf_score'])
tf_df.head(50)

,word,tf_score
0,replace,0.026812
1,repair,0.025386
2,check,0.023443
3,light,0.021635
4,find,0.020558
5,emergency,0.019279
6,srm,0.017115
7,crack,0.016431
8,remove,0.014852
9,door,0.013095


* Delete key words with high term frequency and  not issue-related in stopwords list 

In [72]:
# stopword list updated 
new_st_list = ['replace', 'repair', 'check', 'find', 'remove', 'instal', 'service', 'maintance','flight','aircraft','srm','limit', \
               'normal','iaw','amm','op', 'perform', 'aft','acft','good','ea','note','right','left','fail','inspection','classify',\
               'area','part','operational','maintenance']

## 5. FINAL LDA Model

In [73]:
# update data lemmatized
data_lemmatized_updated = []

for word_list in data_lemmatized:
    in_word_list = []
    for word in word_list:
        if word not in new_st_list:
            in_word_list.append(word) 
    data_lemmatized_updated.append(in_word_list)
print(data_lemmatized_updated)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [74]:
# update id2word, texts, corpus
# Create Dictionary/ID
id2word = corpora.Dictionary(data_lemmatized_updated)

# Create Corpus
texts = data_lemmatized_updated

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [75]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=50000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Compute Perplexity
print('\nPerplexity of year 10-19 dataset:' , lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score year 10-19 dataset:', coherence_lda)

# Print the Keyword in the 20 topics
topic['10-19'] = lda_model.print_topics()
doc_lda = lda_model[corpus]

# Visualize the topics
pyLDAvis.enable_notebook()
vis['10-19'] = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)


Perplexity of year 10-19 dataset: -5.857765616060007

Coherence Score year 10-19 dataset: 0.5464338230690384


In [76]:
topic_df = pd.DataFrame(topic['10-19'], columns=['tpc_id', 'Topic'])#can search the dictionary

In [77]:
get_document_topics = lda_model.get_document_topics(corpus)
print(get_document_topics[0])# get_document_topics extract the topic id for each text records

[(3, 0.5681696), (10, 0.111544974), (15, 0.31341457)]


In [78]:
from operator import itemgetter
clus=[]
for i in range(len(df)):
    x = max(get_document_topics[i],key=itemgetter(1))[0]
    clus.append(x)

In [79]:
clustered = pd.DataFrame(list(zip(clus, text, df.occur_year, df.occur_month,df.c12,df.c18,df.c90,\
                                  df.c120,df.c130,df.manu_model,df.c152,df.c240,df.c250,df.c510f,df.c602)), 
               columns =['tpc_id', 'text', 'year','month','region','segment','name_of_part','manu_name',\
                        'model','manu_model','region_for_aircraft','defective_location','text_failed_part','weight_class',\
                        'wing_type']) 
clustered = pd.merge(clustered, topic_df, on='tpc_id', how='left')

In [80]:
clustered.to_csv("Topic_records_clustered.csv")
topic_df.to_csv('topic_updated.csv')

In [81]:
for i in range(20):
    clustered[clustered.tpc_id == i].to_csv('topic_{}_records.csv'.format(i),index = False)

## 6. New trend of Phrase after stopwords updates

In [83]:
text_per_year = {}
for i in range(10):
    a = []
    for j in list(clustered[clustered.year == 2010+i].index):
        a.append(texts[j])
    text_per_year[2010+i] = a

phrase = []
phrase_id = []
for i in corpus:
    for j in i:
        if '_' in id2word[j[0]]:
            phrase.append(id2word[j[0]])
            phrase_id.append(j[0])

 #Drop duplicate
phrase = list(set(phrase))
phrase_id = list(set(phrase_id))

dic = {}
dic['phrase_id']=phrase_id
dic['phrase_name']=phrase

fre_phrase = pd.DataFrame(dic, columns = ['phrase_id','phrase_name'])
ratio_phrase = fre_phrase

for i in range(10):
    fre_phrase_list = []
    phrase_ratio = []
    for y in phrase:
        fre = 0
        for z in text_per_year[2010+i]:
            for a in list(set(z)):
                if a == y:
                    fre = fre + 1
        fre_phrase_list.append(fre)
        ratio = round(fre/len(text_per_year[2010+i])*10000,2)
        phrase_ratio.append(ratio)
    fre_phrase = pd.concat([fre_phrase, pd.DataFrame(fre_phrase_list,columns = [2010+i])],axis = 1)
    ratio_phrase = pd.concat([ratio_phrase, pd.DataFrame(phrase_ratio,columns = [2010+i])],axis = 1)

total = []
for i in range(fre_phrase.shape[0]):
    total.append(sum(fre_phrase.iloc[i][2:fre_phrase.shape[1]]))

fre_phrase = pd.concat([fre_phrase,pd.DataFrame(total,columns = ['total'])],axis = 1)
fre_phrase.sort_values(by=['total'],ascending=False).head(20)

,phrase_id,phrase_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
3532,6498,approved_return,938,585,403,608,645,343,343,142,115,36,4158
2915,5238,assist_bottle,192,187,160,130,244,200,247,354,351,143,2208
2747,21296,first_officer,85,140,264,218,245,158,139,176,197,41,1663
3848,7087,horizontal_stabilizer,62,81,270,163,248,252,182,109,106,25,1498
990,18168,circuit_breaker,104,125,126,166,117,65,116,208,158,13,1198
3492,22812,without_incident,266,221,90,116,90,108,94,104,89,16,1194
2365,4194,could_duplicate,132,151,124,132,84,84,103,137,151,41,1139
1491,19035,corrective_action,27,38,22,12,33,28,27,89,332,411,1019
866,9787,master_caution,41,51,79,94,111,100,154,156,134,43,963
1850,11515,wheel_well,75,65,94,127,10,46,125,180,145,29,896


In [84]:
index_sorted = list(fre_phrase.sort_values(by=['total'],ascending=False).index)
ratio_phrase.reindex(index_sorted).head(20)# ratio is multiplied by 10000

,phrase_id,phrase_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
3532,6498,approved_return,169.71,85.24,60.20,87.14,110.37,62.16,61.41,26.12,22.16,20.42
2915,5238,assist_bottle,34.74,27.25,23.90,18.63,41.75,36.25,44.22,65.10,67.63,81.13
2747,21296,first_officer,15.38,20.40,39.44,31.24,41.92,28.63,24.89,32.37,37.96,23.26
3848,7087,horizontal_stabilizer,11.22,11.80,40.34,23.36,42.44,45.67,32.58,20.05,20.43,14.18
990,18168,circuit_breaker,18.82,18.21,18.82,23.79,20.02,11.78,20.77,38.25,30.44,7.38
3492,22812,without_incident,48.13,32.20,13.45,16.63,15.40,19.57,16.83,19.13,17.15,9.08
2365,4194,could_duplicate,23.88,22.00,18.52,18.92,14.37,15.22,18.44,25.20,29.10,23.26
1491,19035,corrective_action,4.89,5.54,3.29,1.72,5.65,5.07,4.83,16.37,63.97,233.17
866,9787,master_caution,7.42,7.43,11.80,13.47,18.99,18.12,27.57,28.69,25.82,24.39
1850,11515,wheel_well,13.57,9.47,14.04,18.20,1.71,8.34,22.38,33.10,27.94,16.45


<strong> Output

In [85]:
fre_phrase.sort_values(by=['total'],ascending=False).to_csv('phrase_fre_after_update.csv',index = None)
ratio_phrase.reindex(index_sorted).to_csv('phrase_ratio_after_update.csv',index = None)

## 7.Phrase Analysis

* We select some meaningful phrase to output a file with these phrases and their attributes 

In [86]:
useful_phrase = ['first_officer', 'horizontal_stabilizer','circuit_breaker','master_caution','wheel_well','oxygen_mask',\
                'alodined_prime','tail_cone','boost_pump','landed_safely','extension_retraction','thrust_reverser',\
                'weather_radar','first_clas','excessive_play','appropriate_batterie','life_vest','torque_tube','eddy_current',\
                'lightning_strike','ecam_screen','master_warning','anti_skid','parking_brake','horizontal_stab',\
                'pathway_lighting','bird_strike','land_verify','seal_depressor','fire_extinguisher','dirty_sock',\
                'trapped_water','chip_detector','feeder_cable','satcom_lockup','wheel_steering','icing_condition','horiz_stab',\
                'driven_pump','circuit_breaker_trippe','heat_exchanger','arming_lever','taxied_gate','compressor_stall',\
                'step_title_wing','rub_strip','filter_bypas','coalescer_bag','recirculation_filter','short_final','rivet_head',\
                'drain_mast','recirculation_fan','wire_bundle','speed_tape','skew_sensor','duct_burnout','fan_blade',\
                'rudder_pedal','towed_gate','quick_disconnect','tailcone_release','auto_throttle','airframe_vibration','aural_warning',\
                'pitch_trim','gust_lock','first_class','loud_noise','tail_pipe','clevis_adapter','thrust_lever','life_raft',\
                'plates_skin','screw_strike','oil_consumption','interconnecting_strut','pitot_tube','hepa_filter','vibration_felt',\
                'ozone_filter','prox_sensor','solid_rivet','potable_water','infant_life_vest','water_separator','rotary_actuator',\
                'core_cowl','oversized_bushing','burn_mark']


In [87]:
col = ['phrase_name']
for i in clustered.columns:
    col.append(i)

In [88]:
meaningful_phrase = pd.DataFrame(columns = col)
for i in useful_phrase:
    for j in range(len(data_lemmatized_updated)):
        if i in list(set(data_lemmatized_updated[j])):
            phrase_name = [i]
            row = pd.DataFrame(phrase_name,columns = ['phrase_name'],index = [j])
            records = pd.concat([row,pd.DataFrame([clustered.iloc[j,:]], columns = clustered.columns)],axis =1)
            meaningful_phrase = pd.concat([meaningful_phrase,records],axis = 0)
                

<strong> Output texts with phrase and useful phrase frequency and ratio per year

In [89]:
meaningful_phrase.to_csv('useful_phrase.csv',index = False)

In [90]:
useful_phrase_fre = fre_phrase[fre_phrase.phrase_name.isin(useful_phrase)].sort_values(by = 'total',ascending = False)

In [91]:
useful_phrase_fre.to_csv('useful_phrase_fre.csv',index = False)

In [92]:
ratio_phrase.reindex(useful_phrase_fre.index).to_csv('useful_phrase_ratio.csv',index = False)

## 8. Abbreviation Analysis

* We select some meaningful abbreviation to output a file with these abbreviation and their attributes 

In [93]:
useful_abbr = ['EA','FIG','ARM','RAIL','PAR','FMC','OMEGA','RSS','MEL','ARC','SATCOM','RT','AAR','FCC','ATC','STR',\
              'ASD','MCC']

In [94]:
col = ['abbr_name']
for i in clustered.columns:
    col.append(i)
meaningful_abbr = pd.DataFrame(columns = col)
for i in useful_abbr:
    for j in range(len(data_lemmatized_updated)):
        if i.lower() in list(set(data_lemmatized_updated[j])):
            abbr_name = [i]
            row = pd.DataFrame(abbr_name,columns = ['abbr_name'],index = [j])
            records = pd.concat([row,pd.DataFrame([clustered.iloc[j,:]], columns = clustered.columns)],axis =1)
            meaningful_abbr = pd.concat([meaningful_abbr,records],axis = 0)

<strong> Output texts with abbr and useful abbr frequency and ratio per year

In [95]:
meaningful_abbr.to_csv('useful_abbr.csv',index = False)

In [96]:
useful_abbr_fre = fre_abbr[fre_abbr.abbr_name.isin([x.lower() for x in useful_abbr])].sort_values(by = 'total',ascending = False)

In [97]:
useful_abbr_fre.to_csv('useful_abbr_fre.csv',index = False)

In [98]:
ratio_abbr.reindex(useful_abbr_fre.index).to_csv('useful_abbr_ratio.csv',index = False)

## 9. Top Aircraft models for each topic 

In [99]:
clustered.head(10)

,tpc_id,text,year,month,region,segment,name_of_part,manu_name,model,manu_model,region_for_aircraft,defective_location,text_failed_part,weight_class,wing_type,Topic
0,3,"ON DESCENT TO LAND, PILOT PUT LANDING GEAR HAN...",2010,4,CE,1,RELAY,BEECH,1900,BEECH 1900,CE,LANDING GEAR,STUCK,2.0,L,"0.060*""fault"" + 0.050*""test"" + 0.043*""ground"" ..."
1,14,"DURING SCHEDULED 14Y CHECK, FOUND SKIN DEFORME...",2010,5,NM,1,SKIN,BOEING,737,BOEING 737,NM,FUSELAGE,DEFORMED,2.0,L,"0.172*""crack"" + 0.086*""skin"" + 0.063*""wing"" + ..."
2,5,"DURING SCHEDULED 14Y CHECK, FOUND FWD CARGO FR...",2010,5,NM,1,FRAME,BOEING,737,BOEING 737,NM,FUSELAGE,CRACKED,2.0,L,"0.204*""corrode"" + 0.100*""web"" + 0.070*""shear"" ..."
3,0,"DURING SCHEDULED 14Y CHECK, FOUND L1 DOOR CUTO...",2010,5,NM,1,ANGLE,BOEING,737,BOEING 737,NM,FUSELAGE,CRACKED,2.0,L,"0.073*""hole"" + 0.070*""fit"" + 0.061*""fuel"" + 0...."
4,14,"DURING SCHEDULED 14Y CHECK, FOUND STRINGER CLI...",2010,5,NM,1,STRINGER CLIP,BOEING,737,BOEING 737,NM,FUSELAGE,CRACKED,2.0,L,"0.172*""crack"" + 0.086*""skin"" + 0.063*""wing"" + ..."
5,5,"DURING SCHEDULED 14Y CHECK, FOUND AFT LOWER RT...",2010,5,NM,1,ANGLE,BOEING,737,BOEING 737,NM,FUSELAGE,CORRODED,2.0,L,"0.204*""corrode"" + 0.100*""web"" + 0.070*""shear"" ..."
6,13,AFT GALLEY EMERGENCY EXIT SIGN LENS BROKEN. R...,2010,5,NM,1,SIGN,BOEING,737,BOEING 737,NM,CABIN,BROKEN,2.0,L,"0.154*""exit"" + 0.098*""emergency"" + 0.086*""sign..."
7,17,L1 DOOR EMERGENCY EVAC SLIDE INADVERTENTLY FEL...,2010,5,NaN,4,SLIDE,BOEING,737,BOEING 737,NM,L1 DOOR,OUT OF POSITION,2.0,L,"0.211*""slide"" + 0.058*""position"" + 0.049*""loos..."
8,15,DIVERSION / DECLARED EMERGENCY. CABIN ALTITUD...,2010,5,NM,1,OUTFLOW VALVE,BOEING,737,BOEING 737,NM,CABIN PRESSURE,MALFUNCTIONED,2.0,L,"0.043*""engine"" + 0.019*""valve"" + 0.017*""ground..."
9,11,DIVERSION / DECLARED EMERGENCY. DC FAIL LIGHT...,2010,5,NM,1,BATTERY,BOEING,737,BOEING 737,NM,E&E BAY,FAULTY,2.0,L,"0.124*""emergency"" + 0.111*""light"" + 0.102*""bat..."


In [100]:
model=list(clustered.manu_model.unique())
count=[]
for x in model:
    z=len(df[df['manu_model']==x])
    count.append(z)

In [101]:
model_summary=pd.DataFrame(list(zip(model, count)), columns=['model','count'])

In [102]:
for y in range(20):
    cols=[]
    for x in model:
        eles=len(clustered[(clustered['tpc_id']==y) & (clustered['manu_model']==x)])
        cols.append(eles)
    model_summary[y]=cols

In [103]:
for i in range(20):
    model_summary[i]=list(map(lambda x,y: x/y*100 if y!=0 else 0, model_summary[i],model_summary['count']))

In [104]:
model_summary = model_summary[model_summary['count'] >100 ]

In [105]:
for x in range(20):
    print(f'topic: {x}')
    print(model_summary[['model',x]].sort_values(by=x, ascending=False).head())

topic: 0
           model          0
17      DOUG DC8  11.092715
26  BOMBDR CL600   6.976744
50     DOUG MD10   6.124336
44     DOUG MD90   5.231509
29    AIRBUS 320   4.762527
topic: 1
         model          1
46    DOUG DC6  38.728324
17    DOUG DC8  28.642384
9   BOEING 727  25.755836
33   DOUG MD88  24.212432
44   DOUG MD90  20.525155
topic: 2
           model          2
2      DHAV DHC8  24.953096
39       EMB 120  24.088249
8     BOEING 747  19.091927
19       EMB 135  16.931919
3   CNDAIR CL600  16.803684
topic: 3
         model          3
74  BOEING 787  15.936953
0   BEECH 1900  13.420159
7   BOEING 767  11.978495
4   SAAB SF340  11.334675
1   BOEING 737   9.111397
topic: 4
         model          4
16   DOUG MD11  11.690496
19     EMB 135   7.625995
74  BOEING 787   5.954466
27  EMB ERJ170   5.756201
50   DOUG MD10   5.176797
topic: 5
         model          5
14  LKHEED 382  27.652520
37  BOEING 717  11.965288
9   BOEING 727  11.404516
1   BOEING 737  11.031544
10    DOUG D